In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import re
import matplotlib.transforms as transforms

# A/N: this was developed on JuPyter from the Anaconda 3 suite
# The elaboration of the csv files is carried out by using the Pandas dataframes,
# along with many roundtrips around different data structures such as dictionaries, lists, etc.

Activities = @TODO add file path
Labels = @TODO add file path

# csv's are currently imported by hardcoding the local path into the script.

In [ ]:
logsDf = pd.read_csv(Activities)
logsDf.insert(4, 'Cluster', value = 0, allow_duplicates=False)
labelsDf = pd.read_csv(Labels)
labelsDf

In [ ]:
logsDf['LogName']=logsDf['LogName'].str.replace(r'.xes', '', regex=True)
for i in range(len(logsDf)):
    for j in range(len(labelsDf)):
        if logsDf.at[i, 'LogName'] == labelsDf.at[j, 'LogName']:
            value = labelsDf.at[j, 'ClusterId']
            logsDf.at[i, 'Cluster'] = value
logsDf

In [ ]:
grouped_activities_dict = {}
grouped_activities_dict = logsDf.groupby('Cluster')['Activity'].unique() 
dict((grouped_activities_dict))
for key in grouped_activities_dict.keys():
    grouped_activities_dict[key] = grouped_activities_dict[key]

sets_df = pd.DataFrame(grouped_activities_dict)
sets_df
#to be exact, it's a lists_df


In [ ]:
activities_lists_dict = {}

for i in range (len(sets_df)): # numero cluster: i
    for j in range (len(sets_df.at[i,'Activity'])): #numero attivita: j
        if sets_df.at[i, 'Activity'][j] in activities_lists_dict:
            activities_lists_dict[sets_df.at[i, 'Activity'][j]].append(i)
           
        else:
            activities_lists_dict[sets_df.at[i, 'Activity'][j]]=[] # add the new key x value pair
            activities_lists_dict[sets_df.at[i, 'Activity'][j]].append(i)


            
            
activities_lists_dict

# we currently have a dictionary of activity x (list of clusters)

In [ ]:
activities_lists_dict2 = {}

for k in activities_lists_dict.keys():
    if len(activities_lists_dict[k]) in activities_lists_dict2:
        activities_lists_dict2[len(activities_lists_dict[k])].append(k)
    else:
        activities_lists_dict2[len(activities_lists_dict[k])] = []
        activities_lists_dict2[len(activities_lists_dict[k])].append(k)

activities_lists_dict2 # contains pairs of N x activities which appear in N clusters, now we sort this descending.



In [ ]:
final_activities_list = []
for key in sorted(activities_lists_dict2):
    for a in range(len(activities_lists_dict2[key])):
        final_activities_list = [activities_lists_dict2[key][a]] + final_activities_list

final_activities_list

In [ ]:
df_mapping = pd.DataFrame(final_activities_list)
dict1 = df_mapping.to_dict()
dict2 = dict1[0]


def get_key(val):
    for key, value in dict2.items():
         if val == value:
                return key
 
    return 99

In [ ]:
logsDf['sorting'] = 0
for i in range (len(logsDf)):
    logsDf.at[i, 'sorting'] = get_key(logsDf.at[i, 'Activity'])

logsDf

In [ ]:
logsDf.sort_values(by=['sorting'], inplace=True)
logsDf

In [ ]:
labelsDf

In [ ]:
cluster_edges_list = []
labelsDf.sort_values(by=['ClusterId'], inplace=True)

for i in range(len(labelsDf)-1):
    if labelsDf.at[i, 'ClusterId'] == labelsDf.at[i+1, 'ClusterId']:
        pass
    else:
        cluster_edges_list.append(i+1)


    

cluster_edges_list


In [ ]:
logs_list = labelsDf['LogName']
    
    
all_activities_list = logsDf['Activity'].unique()
output_df = pd.DataFrame(columns=final_activities_list, index=logs_list)
output_df = output_df.fillna(0)
output_df

In [ ]:
for j in range (len(logsDf)):
    if logsDf.at[j,'LogName'].strip("'") in output_df.index and logsDf.at[j, 'Activity'].strip("'") in final_activities_list:
        row = logsDf.at[j,'LogName']
        col = logsDf.at[j, 'Activity']
        if logsDf.at[j, 'IsRepeating'] == 0:
            output_df.loc[row, col] = 0.5 #NOT REPEATING
        else:
            if logsDf.at[j, 'IsRepeating'] == 1:
                output_df.loc[row, col] = 1
output_df

In [ ]:
plt.figure(figsize=(60, 60))
ax = seaborn.heatmap(output_df, cmap="Greens")
# A/N: in this visualization, the color scale which is shown by default on the right side of the heat map
# can very well feel unnecessary: if so, it can be easily omitted by adding cbar=False along with the other
# parameters of seaborn.heatmap()
ax.hlines(cluster_edges_list, *ax.get_xlim()) #adds lines that mark the edges between clusters

ax.tick_params(right=False, top=True, labelright=False, labeltop=True)




plt.show()